In [1]:
from utils.io import gen_id_dated, gdf_viewer, gdf_geom, gdf_merger, na_col_drop, na_line_drop, data_validation
import pandas as pd
import numpy as np
from utils.utils import update_dict

In [2]:
t1 = pd.DataFrame({'ID':['id1','id2','id3','id4','id5','id6','id7','id8'],'Name':['A','B','C','D','E','F','G','H'],
                   'Value':[1,2,2,4,np.nan,5,np.nan,8], 'X': [1.,4.,12.,8.,7.,11.,8.,13.], 'Y': [7.,4.,2.,18.,14.,9.,11.,5.]})
t2 = pd.DataFrame({'ID':['id1','id2','id3','id4','id5','id6','id7','id8'],'Name':['A','B','D','C',np.nan,'F','G','h'],
                   'Value':[1,3,2,2,np.nan,np.nan,6,7],'X': [np.nan,4.,12.,8.,7.,11.,8.,13.], 'Y': [np.nan,4.,2.,18.,14.,9.,1.,5.]})

In [3]:
t1.merge(t2, on='ID', how='outer')

,ID,Name_x,Value_x,X_x,Y_x,Name_y,Value_y,X_y,Y_y
0,id1,A,1.0,1.0,7.0,A,1.0,NaN,NaN
1,id2,B,2.0,4.0,4.0,B,3.0,4.0,4.0
2,id3,C,2.0,12.0,2.0,D,2.0,12.0,2.0
3,id4,D,4.0,8.0,18.0,C,2.0,8.0,18.0
4,id5,E,NaN,7.0,14.0,NaN,NaN,7.0,14.0
5,id6,F,5.0,11.0,9.0,F,NaN,11.0,9.0
6,id7,G,NaN,8.0,11.0,G,6.0,8.0,1.0
7,id8,H,8.0,13.0,5.0,h,7.0,13.0,5.0


In [4]:
m, conflicts = gdf_merger(t1, t2, on='ID', how='outer', dist_max=1.)

Conflict values present. Please resolve this manually !


In [5]:
m

,ID,index,Y,Name,Value,X,split_distinct
0,id1,0.0,7.0,A,1.0,1.0,False
1,id2,1.0,4.0,B,NaN,4.0,False
2,id3,2.0,2.0,#conflict,2.0,12.0,False
3,id4,3.0,18.0,#conflict,NaN,8.0,False
4,id5,4.0,14.0,E,NaN,7.0,False
5,id6,5.0,9.0,F,5.0,11.0,False
6,id8,7.0,5.0,H,NaN,13.0,False
7,_id7_,NaN,11.0,G,NaN,8.0,True
8,id7,NaN,1.0,G,6.0,8.0,True


In [6]:
conflicts

,Check_col,ID,Value_x,Value_y,Name_x,Name_y
1,Value,id2,2.0,3.0,B,B
2,Name,id3,2.0,2.0,C,D
3,"Name, Value",id4,4.0,2.0,D,C
7,Value,id8,8.0,7.0,H,h


In [7]:
data_validation(overall_data=m, conflict_data=conflicts, valid_dict={'Value_x':[1,7], 'Name_y':[3]})

Validation done, but conflicts remain!


In [8]:
m

,ID,index,Y,Name,Value,X,split_distinct
0,id1,0.0,7.0,A,1.0,1.0,False
1,id2,1.0,4.0,B,2.0,4.0,False
2,id3,2.0,2.0,#conflict,2.0,12.0,False
3,id4,3.0,18.0,C,NaN,8.0,False
4,id5,4.0,14.0,E,NaN,7.0,False
5,id6,5.0,9.0,F,5.0,11.0,False
6,id8,7.0,5.0,H,NaN,13.0,False
7,_id7_,NaN,11.0,G,NaN,8.0,True
8,id7,NaN,1.0,G,6.0,8.0,True


In [9]:
conflicts

,Check_col,ID,Value_x,Value_y,Name_x,Name_y
2,Name,id3,2.0,2.0,C,D


In [10]:
m.loc[[1,6], 'Value'] = [1,5]
m.loc[[1,6], 'Value']

1    1.0
6    5.0
Name: Value, dtype: float64